In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def firstFit(result, item):
    #print('firstFit')
    allocated = False
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            result[i] += item
            allocated = True
            break
        
    if not allocated:
        result.append(item)
        
    return result

In [3]:
def nextFit(result, item):
    #print('nextFit')
    allocated = False
    if len(result) > 0:
        bin = result[len(result) - 1]
        if bin + item <= capasity:
            result[len(result) - 1] += item
            allocated = True
        
    if not allocated:
        result.append(item)
        
    return result

In [4]:
def bestFit(result, item):
    #print('bestFit')
    best = float('inf')
    pos = None
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            space = capasity - (bin + item)
            if space < best:
                best = space
                pos = i
                break
        
    if pos == None:
        result.append(item)
    else:
        result[pos] += item
        
    return result

In [5]:
def worstFit(result, item):
    #print('worstFit')
    worst = -1
    pos = None
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            space = capasity - (bin + item)
            if space > worst:
                worst = space
                pos = i
                break
        
    if pos == None:
        result.append(item)
    else:
        result[pos] += item
        
    return result

In [6]:
def fitness(result):
    return np.sum(np.square(np.array(result) / capasity)) / len(result) 

In [7]:
def applyChromosome(items, chromosome):
    result = []
    idx = 0
    while idx < len(items):
        for heuristic in chromosome:
            if idx < len(items):
                heuristic(result, items[idx])
                idx += 1
                #print(result)
            else:
                break
            
    return result

In [8]:
def generateChromosome():
    chromosome = []
    chromosomeLength = np.random.randint(low=MIN_CHROMOSOME_SIZE,high=MAX_CHROMOSOME_SIZE)
    for i in range(chromosomeLength):
        heuristicChoice = np.random.randint(len(heuristicList))
        chromosome.append(heuristicList[heuristicChoice])
        
    return chromosome

In [9]:
def createPopulation():
    population = []
    for i in range(POPULATION_SIZE):
        population.append(generateChromosome())
        
    return population

In [10]:
def getPopulationFitness(population):
    populationResults = []
    for individual in population:
        populationResults.append(applyChromosome(items, individual))
        
    populationFitness = []
    for result in populationResults:
        populationFitness.append([fitness(result), len(result)])
        
    return populationFitness, populationResults

In [11]:
def swap(chromosome, times=1):
    clone = chromosome[:]
    
    for _ in range(times):
        pos1 = np.random.randint(len(clone))
        pos2 = np.random.randint(len(clone))
        tmp = clone[pos1]
        clone[pos1] = clone[pos2]
        clone[pos2] = tmp
        
    return clone

In [12]:
def tournamentSelector(population, tournament_size=5):
    # Make the tournament
    random_indicies = np.random.randint(len(population), size=tournament_size).tolist()
    tournament = []
    for idx, val in np.ndenumerate(random_indicies):
        tournament.append(population[val])

    # Run the tournament
    fitnesss, results = getPopulationFitness(tournament)
    
    maxPos = np.argmax(fitnesss, axis=0)[0]
    minPos = np.argmin(fitnesss, axis=0)[0]
    
    return [random_indicies[maxPos], fitnesss[maxPos]], [random_indicies[minPos], fitnesss[minPos]]

In [30]:
location = '../experiments/problems/'

problems = [
            ['bin1data/N1C1W1_A.BPP', 25],
            ['bin1data/N1C1W1_O.BPP', 32],
            ['bin1data/N1C1W1_P.BPP', 26],
            ['bin1data/N1C1W1_T.BPP', 28],
            ['bin1data/N1C1W1_G.BPP', 25],
            ['bin1data/N1C1W1_K.BPP', 26],
            ['bin1data/N1C1W1_S.BPP', 28],
            ['bin1data/N1C1W1_F.BPP', 27],
    
            ['bin2data/N1W1B1R0.BPP', 18],
            ['bin2data/N1W1B1R1.BPP', 18],
            ['bin2data/N1W2B3R2.BPP', 10]
        ]

heuristicList = [
    firstFit,
    nextFit,
    bestFit,
    worstFit
]

problemIndex = 8
problemSet = pd.read_csv(location + problems[problemIndex][0], header=None).values.tolist()

PROBLEM_SIZE = problemSet.pop(0)[0]
BIN_CAPACITY = problemSet.pop(0)[0]

items = pd.DataFrame(problemSet)
items = np.array(items[0])

# items = [5, 3, 7, 2, 9, 4, 8, 9, 3, 5, 5, 3, 7, 2, 9, 4, 8, 9, 3, 5]
assert PROBLEM_SIZE == len(items)

capasity = BIN_CAPACITY

MIN_CHROMOSOME_SIZE = 5
MAX_CHROMOSOME_SIZE = PROBLEM_SIZE

POPULATION_SIZE = 50
GENERATIONS = 10000

population = createPopulation()
#population = [[heuristicList[0]], [heuristicList[1]], [heuristicList[2]], [heuristicList[3]]]

endResult = []

for gen in range(GENERATIONS):    
    best, worst = tournamentSelector(population, tournament_size=6)
    if gen % 1 == 0:
        fitnesss, results = getPopulationFitness(population)
        max = np.argmax(fitnesss, axis=0)
        endResult.append(results[max[0]])
        if problems[problemIndex][1] == fitnesss[max[0]][1]:
            print('=== Found known min: ', problems[problemIndex][1], '===')
            break
        print(gen)
        print(fitnesss[max[0]], results[max[0]])
        print('###')
#         for i, fit in enumerate(fitnesss):
#             print(population[i])
#             print(fitnesss[i], results[i])
    
    population[worst[0]] = swap(population[best[0]], 1)


0
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
1
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
2
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
3
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
4
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
5
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
6
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
7
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
8
[0.744311, 20]

76
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
77
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
78
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
79
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
80
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
81
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
82
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
83
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
84
[0.74

155
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
156
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
157
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
158
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
159
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
160
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
161
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
162
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###


243
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
244
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
245
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
246
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
247
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
248
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
249
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
250
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###


340
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
341
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
342
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
343
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
344
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
345
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
346
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
347
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###


433
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
434
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
435
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
436
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
437
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
438
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
439
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###
440
[0.744311, 20] [789, 785, 779, 772, 765, 759, 747, 733, 998, 1000, 998, 997, 992, 998, 985, 936, 897, 883, 846, 275]
###


KeyboardInterrupt: 

In [14]:
test = np.array(endResult)

In [15]:
np.sum(items)



16946

In [19]:
np.sum(endResult[len(endResult) - 1])

16946